### NOTEBOOK SPARK FEITO NO GOOGLE COLAB

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!pip install -q pyspark

In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import pyspark.sql.functions as f
import pyspark.sql.types as t

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

In [ ]:
url = "https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("titanic.csv"), header=True, sep=';', inferSchema=True)

In [ ]:
#QUESTÃO 1

df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
#QUESTÃO 2

df.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
#QUESTÃO 3

df.filter(df['Survived'] == 1).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+--------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599| 71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|    53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742| 11.1333| null|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736| 30.0708| null|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    

In [ ]:
#QUESTÃO 4
 
df.filter(df['Survived'] == 0).filter(df['sex'] == 'male').show()

+-----------+--------+------+--------------------+----+----+-----+-----+---------------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|         Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------------+-------+-----------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|      A/5 21171|   7.25|       null|       S|
|          5|       0|     3|Allen, Mr. Willia...|male|35.0|    0|    0|         373450|   8.05|       null|       S|
|          6|       0|     3|    Moran, Mr. James|male|null|    0|    0|         330877| 8.4583|       null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim...|male|54.0|    0|    0|          17463|51.8625|        E46|       S|
|          8|       0|     3|Palsson, Master. ...|male| 2.0|    3|    1|         349909| 21.075|       null|       S|
|         13|       0|     3|Saundercock, Mr. ...|male|2

In [ ]:
#QUESTÃO 5

df.groupBy('Pclass').agg(f.mean('Fare')).show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.15468749999992|
|     3|13.675550101832997|
|     2| 20.66218315217391|
+------+------------------+



In [ ]:
#QUESTÃO 6

df.groupBy('Pclass', 'sex').agg(f.mean('Fare')).show()

+------+------+------------------+
|Pclass|   sex|         avg(Fare)|
+------+------+------------------+
|     2|female| 21.97012105263158|
|     3|  male|12.661632564841513|
|     1|  male| 67.22612704918033|
|     3|female|16.118809722222224|
|     1|female|106.12579787234041|
|     2|  male| 19.74178240740741|
+------+------+------------------+



In [ ]:
#QUESTÃO 7

df.filter(df['Survived'] == 1).groupBy('Pclass', 'sex', 'Survived').count().show()

+------+------+--------+-----+
|Pclass|   sex|Survived|count|
+------+------+--------+-----+
|     3|female|       1|   72|
|     2|female|       1|   70|
|     2|  male|       1|   17|
|     3|  male|       1|   47|
|     1|female|       1|   91|
|     1|  male|       1|   45|
+------+------+--------+-----+



In [ ]:
#QUESTÃO 8

def Age_category(Age):
  if Age is None:
    return "null"
  elif Age < 18.0:
    return "Criança"
  elif ((Age >= 18.0) and (Age <= 40.0)):
    return "Adulto Jovem"
  else:
    return "Adulto Maduro"

Age_udf=udf(Age_category, StringType())
df=df.withColumn('Age_category',Age_udf('Age'))
df.filter(df['Survived'] == 1).groupBy('Age_category', 'sex', 'Survived').count().show()

+-------------+------+--------+-----+
| Age_category|   sex|Survived|count|
+-------------+------+--------+-----+
|         null|female|       1|   36|
|      Criança|  male|       1|   23|
|      Criança|female|       1|   38|
| Adulto Jovem|  male|       1|   52|
|Adulto Maduro|female|       1|   37|
|Adulto Maduro|  male|       1|   18|
|         null|  male|       1|   16|
| Adulto Jovem|female|       1|  122|
+-------------+------+--------+-----+



In [ ]:
#QUESTÃO 9 - Escrevendo dados em Parquet

df.filter(df['Survived'] == 1).write.partitionBy('sex', 'Pclass').parquet("/Titanic.parquet")